In [4]:
import pandas as pd
data = pd.read_csv('traindata2')

In [5]:
alltext = []
for i in data['text']:
    tl = i.split(' ')
    alltext.append(tl)

In [6]:
from gensim.corpora import Dictionary
dct = Dictionary(alltext)

In [7]:
from keras.preprocessing import sequence
X = [dct.doc2idx(txt) for txt in alltext]
# X =sequence.pad_sequences(X, padding='post', value=len(dct))

In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test= train_test_split(X,data.iloc[:,-1],test_size=0.3,random_state=0)

In [9]:
X_train_padded_seqs = sequence.pad_sequences(X_train, padding='post', maxlen=max([len(x) for x in X]), value=len(dct))
X_test_padded_seqs = sequence.pad_sequences(X_test, padding='post', maxlen=max([len(x) for x in X]), value=len(dct))

left_train_word_ids = [[len(dct)] + x[:-1] for x in X_train]
left_test_word_ids = [[len(dct)] + x[:-1] for x in X_test]
right_train_word_ids = [x[1:] + [len(dct)] for x in X_train]
right_test_word_ids = [x[1:] + [len(dct)] for x in X_test]

# 分別對左邊和右邊的詞進行編碼
left_train_padded_seqs = sequence.pad_sequences(left_train_word_ids, padding='post', maxlen=max([len(x) for x in X]), value=len(dct))
left_test_padded_seqs = sequence.pad_sequences(left_test_word_ids, padding='post', maxlen=max([len(x) for x in X]), value=len(dct))
right_train_padded_seqs = sequence.pad_sequences(right_train_word_ids, padding='post', maxlen=max([len(x) for x in X]), value=len(dct))
right_test_padded_seqs = sequence.pad_sequences(right_test_word_ids, padding='post', maxlen=max([len(x) for x in X]), value=len(dct))

In [10]:
from keras.layers import Input
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers.merge import concatenate
from keras.layers import TimeDistributed
from keras.layers import Lambda
from keras import backend
from keras.layers import Dense
from keras.models import Model

In [11]:
# 模型共有三個輸入，分別是左詞，右詞和中心詞
document = Input(shape = (None, ))
left_context = Input(shape = (None, ))
right_context = Input(shape = (None, ))

# 構建詞向量
embedder = Embedding(len(dct) + 1, 300, input_length = max([len(x) for x in X]))
doc_embedding = embedder(document)
l_embedding = embedder(left_context)
r_embedding = embedder(right_context)

forward = LSTM(256, return_sequences = True)(l_embedding) 
backward = LSTM(256, return_sequences = True, go_backwards = True)(r_embedding) 
together = concatenate([forward, doc_embedding, backward], axis = 2)

semantic = TimeDistributed(Dense(128, activation = "tanh"))(together)

pool_rnn = Lambda(lambda x: backend.max(x, axis = 1), output_shape = (128, ))(semantic) 
output = Dense(4, activation = "softmax")(pool_rnn)
model = Model(inputs = [document, left_context, right_context], outputs = output)
model.summary()
model.compile(loss="sparse_categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_6 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 300)    8018400     ['input_4[0][0]',                
                                                                  'input_5[0][0]',            

In [12]:
model.fit([X_train_padded_seqs, left_train_padded_seqs, right_train_padded_seqs], y_train, validation_split=0.1, epochs=10, verbose=1)

Epoch 1/10
9/9 [==============================] - 273s 31s/step - loss: 1.3947 - accuracy: 0.2529 - val_loss: 1.3957 - val_accuracy: 0.1667
Epoch 2/10
9/9 [==============================] - 292s 32s/step - loss: 1.2838 - accuracy: 0.6169 - val_loss: 1.3777 - val_accuracy: 0.2667
Epoch 3/10
9/9 [==============================] - 290s 32s/step - loss: 1.1749 - accuracy: 0.8276 - val_loss: 1.3721 - val_accuracy: 0.2000
Epoch 4/10
9/9 [==============================] - 591s 69s/step - loss: 1.0445 - accuracy: 0.8008 - val_loss: 1.6666 - val_accuracy: 0.2667
Epoch 5/10
9/9 [==============================] - 291s 32s/step - loss: 1.1499 - accuracy: 0.6092 - val_loss: 1.3230 - val_accuracy: 0.4667
Epoch 6/10
9/9 [==============================] - 291s 32s/step - loss: 0.8547 - accuracy: 0.9693 - val_loss: 1.3477 - val_accuracy: 0.4000
Epoch 7/10
9/9 [==============================] - 310s 34s/step - loss: 0.6722 - accuracy: 0.9885 - val_loss: 1.3898 - val_accuracy: 0.3000
Epoch 8/10
9/9 [====

In [12]:
model.evaluate([X_test_padded_seqs, left_test_padded_seqs, right_test_padded_seqs], y_test ,batch_size=10)

13/13 [==============================] - 11s 852ms/step - loss: 1.9261 - accuracy: 0.1920


[1.9261211156845093, 0.19200000166893005]

In [13]:
model.predict([X_test_padded_seqs, left_test_padded_seqs, right_test_padded_seqs])

array([[0.3506575 , 0.2043105 , 0.2936441 , 0.15138793],
       [0.28047124, 0.6399319 , 0.03927177, 0.04032504],
       [0.0954985 , 0.07138278, 0.60311   , 0.23000872],
       [0.20593175, 0.13669425, 0.15304764, 0.50432634],
       [0.44682813, 0.5175354 , 0.01896965, 0.01666683],
       [0.24265702, 0.39784387, 0.12719049, 0.23230861],
       [0.12921044, 0.07024639, 0.35202482, 0.4485184 ],
       [0.43140352, 0.52141434, 0.01785414, 0.02932801],
       [0.18947117, 0.1799854 , 0.34334958, 0.28719386],
       [0.28650007, 0.13657309, 0.28635538, 0.29057145],
       [0.10012407, 0.08231197, 0.38097516, 0.43658876],
       [0.16419315, 0.09561237, 0.39145693, 0.34873757],
       [0.19202197, 0.16564211, 0.3191769 , 0.32315907],
       [0.19305083, 0.7717174 , 0.02621903, 0.00901268],
       [0.09762999, 0.07569057, 0.5960233 , 0.23065616],
       [0.4938917 , 0.34562767, 0.0866355 , 0.07384519],
       [0.14206946, 0.15542956, 0.5223737 , 0.18012732],
       [0.4277156 , 0.4701517 ,

In [ ]:
from keras.models import load_model
model.save('kerassave')
newmodel = load_model('kerassave')